In [3]:
import requests
import json
import base64
import PIL
from PIL import Image, ImageOps
import pandas as pd
import time
import statsmodels.api as sm

In [10]:

BASE_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'

def detect(image_path):
    """
    This function uses HTTP POST to query the Face++ API 
    that detects and classifies faces in the given image
    
    Args:
        image_path: path to the image that we want to classify
        
    Returns:
        result: a dictionary containing the API response
    """

    #read the image
    im = Image.open(image_path)
    #flip image
    out = ImageOps.grayscale(im)
    split = image_path.split("/")
    split[-1] = "grayscale-" + split[-1]
    split[-2] = "grayscale"
    transp_image_path = "/".join(split)
    out.save(transp_image_path)
    
    
    def get_call(image_path): 
        with open(image_path, 'rb') as image_file:
            # encode in base64
            image_base64 = base64.b64encode(image_file.read())
            
            return {'api_key': "AJUC6jZcYaEtVm62lA944PD18g8qzLn-",
                'api_secret': "NiML8ojHjroKMog2AKvzMuu5PpaJhDP2",
                'image_base64': image_base64,
                'return_attributes': 'gender,age'} 
    
    
    original_response = requests.post(url = BASE_URL, data = get_call(image_path))
    flipped_response = requests.post(url = BASE_URL, data = get_call(transp_image_path))
    return json.loads(original_response.content), json.loads(flipped_response.content)

In [13]:
def hit_fpp_api(directory):
    original_ages = []
    original_genders = []
    transformed_ages = []
    transformed_genders = []
    images = []
    for image in os.listdir(f"{directory}/original"):
        if image != ".DS_Store":
            images.append(image)
            original, flipped  = detect(f"{directory}/original/{image}")
            original_ages.append(original['faces'][0]['attributes']['age']['value'])
            original_genders.append(original['faces'][0]['attributes']['gender']['value'])
            transformed_ages.append(flipped['faces'][0]['attributes']['age']['value'])
            transformed_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
            time.sleep(2)
    df = pd.DataFrame()
    df["Images"] = images
    df["Original Age"] = original_ages
    df["Original Gender"] = original_genders
    df["Y-Axis Flip Age"] = transformed_ages
    df["Y-Axis Flip Gender"] = transformed_genders
    return df

In [14]:
style_gan_df = hit_fpp_api("../stock_images")

In [15]:
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,c_2_0_3.jpg,36,Female,31,Male
1,c_2_2_1.jpg,22,Female,25,Male
2,d_0_0_2.jpg,48,Female,54,Female
3,e_2_0_4.jpg,80,Female,58,Female
4,d_0_2_0.jpg,16,Male,46,Female
...,...,...,...,...,...
95,a_0_2_1.jpg,21,Male,23,Male
96,a_0_2_3.jpg,53,Male,52,Male
97,a_0_0_1.jpg,25,Female,24,Female
98,a_0_0_0.jpg,34,Female,40,Female


In [19]:
style_gan_df.to_csv("grayscale_stylegan_data.csv")

In [16]:
stock_df = hit_fpp_api("../stylegan_images")

In [17]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,1628188590507_2_2_3.jpg,66,Male,65,Male
1,1628188590507_2_0_1.jpg,21,Female,21,Female
2,1628188590507_2_0_0.jpg,20,Female,20,Female
3,1628188590507_2_2_2.jpg,31,Male,30,Male
4,1628188590507_2_2_0.jpg,19,Female,19,Female
...,...,...,...,...,...
95,1628188536814_0_2_4.jpg,80,Male,80,Male
96,1628188536814_0_0_4.jpg,83,Female,82,Female
97,1628188755886_2_0_4.jpg,75,Female,68,Female
98,1628188704081_2_0_4.jpg,82,Male,76,Female


In [20]:
stock_df.to_csv("grayscale_stock_data.csv")

In [21]:
def add_col(df):
    df["Age Difference"] = df["Original Age"] / df["Y-Axis Flip Age"]
    original_gender_to_int = [1 if x == "Male" else -1 for x in df["Original Gender"]]
    flip_gender_to_int = [-1 if x == "Male" else 1 for x in df["Y-Axis Flip Gender"]]
    df["Gender Difference"] = [a + b for a,b in zip(original_gender_to_int, flip_gender_to_int)]
    return df
style_gan_df = add_col(style_gan_df)
stock_df = add_col(stock_df)

In [22]:
style_gan_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,c_2_0_3.jpg,36,Female,31,Male,1.161290,-2
1,c_2_2_1.jpg,22,Female,25,Male,0.880000,-2
2,d_0_0_2.jpg,48,Female,54,Female,0.888889,0
3,e_2_0_4.jpg,80,Female,58,Female,1.379310,0
4,d_0_2_0.jpg,16,Male,46,Female,0.347826,2
...,...,...,...,...,...,...,...
95,a_0_2_1.jpg,21,Male,23,Male,0.913043,0
96,a_0_2_3.jpg,53,Male,52,Male,1.019231,0
97,a_0_0_1.jpg,25,Female,24,Female,1.041667,0
98,a_0_0_0.jpg,34,Female,40,Female,0.850000,0


In [23]:
stock_df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender,Age Difference,Gender Difference
0,1628188590507_2_2_3.jpg,66,Male,65,Male,1.015385,0
1,1628188590507_2_0_1.jpg,21,Female,21,Female,1.000000,0
2,1628188590507_2_0_0.jpg,20,Female,20,Female,1.000000,0
3,1628188590507_2_2_2.jpg,31,Male,30,Male,1.033333,0
4,1628188590507_2_2_0.jpg,19,Female,19,Female,1.000000,0
...,...,...,...,...,...,...,...
95,1628188536814_0_2_4.jpg,80,Male,80,Male,1.000000,0
96,1628188536814_0_0_4.jpg,83,Female,82,Female,1.012195,0
97,1628188755886_2_0_4.jpg,75,Female,68,Female,1.102941,0
98,1628188704081_2_0_4.jpg,82,Male,76,Female,1.078947,2


In [24]:
stylegan_changed_age = (style_gan_df["Original Age"] != style_gan_df["Y-Axis Flip Age"]).mean()
stylegan_changed_age

0.88

In [47]:
stylegan_year_change = (abs(style_gan_df["Original Age"] - style_gan_df["Y-Axis Flip Age"]) > 5).mean()
stylegan_year_change

0.26

In [46]:
stylegan_changed_gender = (style_gan_df["Original Gender"] != style_gan_df["Y-Axis Flip Gender"]).mean()
stylegan_changed_gender

0.06

In [50]:
stock_changed_age = (stock_df["Original Age"] != stock_df["Y-Axis Flip Age"]).mean()
stock_changed_age

0.15

In [48]:
stock_year_change = (abs(stock_df["Original Age"] - stock_df["Y-Axis Flip Age"]) > 5).mean()
stock_year_change

0.29

In [51]:
stock_changed_gender = (stock_df["Original Gender"] != stock_df["Y-Axis Flip Gender"]).mean()
stock_changed_gender

0.04

In [28]:
def get_dems_of_row(row_name):
    if row_name[-9] == "2":
        race = "black"
    else:
        race = "white"
    if row_name[-7] == "2":
        gender = " male"
    else:
        gender = " female"
    if row_name[-5] == "0":
        age = "child"
    elif row_name[-5] == "1":
        age = "teen"
    elif row_name[-5] == "2":
        age = "adult"
    elif row_name[-5] == "3":
        age = "middle-age"
    else:
        age = "old"
    return gender + " " + race + " " + age


def rename_index(df):
    index = []
    for ad in pd.Series(df["Images"]):
        new_name = get_dems_of_row(ad)
        index.append(new_name)

    df["Images"] = index

rename_index(style_gan_df)
rename_index(stock_df)

In [29]:
def determine_true_age(df):
    def age_bucket(age_str):
        if "child" in age_str:
            return "0-12"
        elif "teen" in age_str:
            return "12-18"
        elif "adult" in age_str:
            return "19-34"
        elif "middle-age" in age_str:
            return "35-54"
        else:
            return "65+"
    df["True Age"] = df["Images"].map(age_bucket)
    return df

stock_df = determine_true_age(stock_df)
style_gan_df = determine_true_age(style_gan_df)

        

In [80]:
def determine_true_gender(df):
    def gender(g_str):
        if ("female" in g_str):
            return "Female"
        else:
            return "Male"
    df["True Gender"] = df["Images"].map(gender)
    return df

stock_df = determine_true_gender(stock_df)
style_gan_df = determine_true_gender(style_gan_df)

In [31]:
def generate_age_buckets(df):
    def age_bucket(num):
        if num < 12:
            return "0-12"
        elif num < 18:
            return "12-18"
        elif num < 34:
            return "19-34"
        elif num < 54:
            return "35-54"
        else:
            return "65+"
    df["Original Age Bucket"] = df["Original Age"].map(age_bucket)
    df["Y-Axis Flip Age Bucket"] = df["Y-Axis Flip Age"].map(age_bucket)
    return df

stock_df = generate_age_buckets(stock_df)
style_gan_df = generate_age_buckets(style_gan_df)

In [81]:
stock_df

,Images,Original Age,Original Gender,Grayscale Age,Grayscale Gender,Age Difference,Gender Difference,True Age,Original Age Bucket,Grayscale Age Bucket,True Gender
0,male black middle-age,66,Male,65,Male,1.015385,0,35-54,65+,65+,Male
1,female black teen,21,Female,21,Female,1.000000,0,12-18,19-34,19-34,Female
2,female black child,20,Female,20,Female,1.000000,0,0-12,19-34,19-34,Female
3,male black adult,31,Male,30,Male,1.033333,0,19-34,19-34,19-34,Male
4,male black child,19,Female,19,Female,1.000000,0,0-12,19-34,19-34,Male
...,...,...,...,...,...,...,...,...,...,...,...
95,male white old,80,Male,80,Male,1.000000,0,65+,65+,65+,Male
96,female white old,83,Female,82,Female,1.012195,0,65+,65+,65+,Female
97,female black old,75,Female,68,Female,1.102941,0,65+,65+,65+,Female
98,female black old,82,Male,76,Female,1.078947,2,65+,65+,65+,Female


In [61]:
stock_df = stock_df.rename(columns = {"Y-Axis Flip Age": "Grayscale Age", "Y-Axis Flip Gender": "Grayscale Gender", "Y-Axis Flip Age Bucket": "Grayscale Age Bucket"})
stock_df

,Images,Original Age,Original Gender,Grayscale Age,Grayscale Gender,Age Difference,Gender Difference,True Age,Original Age Bucket,Grayscale Age Bucket
0,male black middle-age,66,Male,65,Male,1.015385,0,35-54,65+,65+
1,female black teen,21,Female,21,Female,1.000000,0,12-18,19-34,19-34
2,female black child,20,Female,20,Female,1.000000,0,0-12,19-34,19-34
3,male black adult,31,Male,30,Male,1.033333,0,19-34,19-34,19-34
4,male black child,19,Female,19,Female,1.000000,0,0-12,19-34,19-34
...,...,...,...,...,...,...,...,...,...,...
95,male white old,80,Male,80,Male,1.000000,0,65+,65+,65+
96,female white old,83,Female,82,Female,1.012195,0,65+,65+,65+
97,female black old,75,Female,68,Female,1.102941,0,65+,65+,65+
98,female black old,82,Male,76,Female,1.078947,2,65+,65+,65+


In [62]:
style_gan_df = style_gan_df.rename(columns = {"Y-Axis Flip Age": "Grayscale Age", "Y-Axis Flip Gender": "Grayscale Gender", "Y-Axis Flip Age Bucket": "Grayscale Age Bucket"})
style_gan_df

,Images,Original Age,Original Gender,Grayscale Age,Grayscale Gender,Age Difference,Gender Difference,True Age,Original Age Bucket,Grayscale Age Bucket
0,female black middle-age,36,Female,31,Male,1.161290,-2,35-54,35-54,19-34
1,male black teen,22,Female,25,Male,0.880000,-2,12-18,19-34,19-34
2,female white adult,48,Female,54,Female,0.888889,0,19-34,35-54,65+
3,female black old,80,Female,58,Female,1.379310,0,65+,65+,65+
4,male white child,16,Male,46,Female,0.347826,2,0-12,12-18,35-54
...,...,...,...,...,...,...,...,...,...,...
95,male white teen,21,Male,23,Male,0.913043,0,12-18,19-34,19-34
96,male white middle-age,53,Male,52,Male,1.019231,0,35-54,35-54,35-54
97,female white teen,25,Female,24,Female,1.041667,0,12-18,19-34,19-34
98,female white child,34,Female,40,Female,0.850000,0,0-12,35-54,35-54


In [65]:
style_gan_df.to_csv("../greyscale/grayscale_stylegan_data_analysis.csv")

In [66]:
style_gan_df = pd.read_csv("../greyscale/grayscale_stylegan_data_analysis.csv", index_col=0)
stock_df = pd.read_csv("../greyscale/grayscale_stock_data_analysis.csv", index_col=0)

In [82]:
stock_df

,Images,Original Age,Original Gender,Grayscale Age,Grayscale Gender,Age Difference,Gender Difference,True Age,Original Age Bucket,Grayscale Age Bucket,True Gender
0,male black middle-age,66,Male,65,Male,1.015385,0,35-54,65+,65+,Male
1,female black teen,21,Female,21,Female,1.000000,0,12-18,19-34,19-34,Female
2,female black child,20,Female,20,Female,1.000000,0,0-12,19-34,19-34,Female
3,male black adult,31,Male,30,Male,1.033333,0,19-34,19-34,19-34,Male
4,male black child,19,Female,19,Female,1.000000,0,0-12,19-34,19-34,Male
...,...,...,...,...,...,...,...,...,...,...,...
95,male white old,80,Male,80,Male,1.000000,0,65+,65+,65+,Male
96,female white old,83,Female,82,Female,1.012195,0,65+,65+,65+,Female
97,female black old,75,Female,68,Female,1.102941,0,65+,65+,65+,Female
98,female black old,82,Male,76,Female,1.078947,2,65+,65+,65+,Female


In [83]:
def get_acc(df):
    return (df["Grayscale Age Bucket"] == df["True Age"]).mean() - (df["Original Age Bucket"] == df["True Age"]).mean()

In [84]:
def get_gender_acc(df):
    return (df["Grayscale Gender"] == df["True Gender"]).mean() - (df["Original Gender"] == df["True Gender"]).mean()

In [88]:
def analysis(df, year_threshold = 5):
    changed_age = (df["Original Age"] != df["Grayscale Age"]).mean()
    year_change = (abs(df["Original Age"] - df["Grayscale Age"]) > year_threshold).mean()
    changed_gender = (df["Original Gender"] != df["Grayscale Gender"]).mean()
    acc = get_acc(df)
    gender_acc = get_gender_acc(df)
    print("Changed Age: ", changed_age)
    print("Year over threshold: ", year_change)
    print("Changed Gender: ", changed_gender)
    print("Overall accuracy: ", acc)
    print("Overall gender accuracy: ", gender_acc)

In [89]:
analysis(stock_df)

Changed Age:  0.85
Year over threshold:  0.29
Changed Gender:  0.04
Overall accuracy:  0.040000000000000036
Overall gender accuracy:  0.0


In [90]:
analysis(style_gan_df)

Changed Age:  0.88
Year over threshold:  0.26
Changed Gender:  0.06
Overall accuracy:  -0.010000000000000009
Overall gender accuracy:  -0.020000000000000018


In [91]:
style_gan_df.describe()

,Original Age,Grayscale Age,Age Difference,Gender Difference
count,100.000000,100.000000,100.000000,100.000000
mean,40.010000,38.680000,1.020434,-0.080000
std,21.500526,18.059878,0.183593,0.485757
min,6.000000,10.000000,0.347826,-2.000000
25%,22.000000,23.000000,0.943860,0.000000
50%,32.500000,34.000000,1.032888,0.000000
75%,54.750000,54.000000,1.100000,0.000000
max,91.000000,81.000000,1.577778,2.000000


In [92]:
stock_df.describe()

,Original Age,Grayscale Age,Age Difference,Gender Difference
count,100.000000,100.000000,100.000000,100.00
mean,45.060000,42.680000,1.038836,0.04
std,26.041079,23.540157,0.161322,0.40
min,6.000000,13.000000,0.421053,-2.00
25%,20.750000,20.000000,1.000000,0.00
50%,36.500000,32.500000,1.049390,0.00
75%,73.250000,68.250000,1.114286,0.00
max,87.000000,87.000000,1.717391,2.00


In [93]:
black_stock_df = stock_df.loc[stock_df["Images"].str.contains("black")].reset_index()
white_stock_df = stock_df.loc[stock_df["Images"].str.contains("white")].reset_index()
black_stylegan_df = stock_df.loc[style_gan_df["Images"].str.contains("black")].reset_index()
white_stylegan_df = stock_df.loc[style_gan_df["Images"].str.contains("white")].reset_index()

In [94]:
analysis(black_stock_df)


Changed Age:  0.88
Year over threshold:  0.44
Changed Gender:  0.04
Overall accuracy:  0.09999999999999998
Overall gender accuracy:  0.0


In [95]:
analysis(white_stock_df)

Changed Age:  0.82
Year over threshold:  0.14
Changed Gender:  0.04
Overall accuracy:  -0.019999999999999962
Overall gender accuracy:  0.0


In [96]:

analysis(black_stylegan_df)

Changed Age:  0.88
Year over threshold:  0.22
Changed Gender:  0.06
Overall accuracy:  0.03999999999999998
Overall gender accuracy:  -0.020000000000000018


In [97]:

analysis(white_stylegan_df)

Changed Age:  0.82
Year over threshold:  0.36
Changed Gender:  0.02
Overall accuracy:  0.040000000000000036
Overall gender accuracy:  0.020000000000000018
